In [2]:
import pandas as pd

In [ ]:
# load all tables

In [19]:
employeeDF = pd.read_csv("../data/raw/Employee.csv")
education_levelDF = pd.read_csv("../data/raw/EducationLevel.csv")
performanceDF = pd.read_csv("../data/raw/PerformanceRating.csv")
rating_levelDF = pd.read_csv("../data/raw/RatingLevel.csv")
satisfaction_levelDF = pd.read_csv("../data/raw/SatisfiedLevel.csv")

In [ ]:
# cleaning the columns names

In [46]:
def clean_columns(df):
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace("(", "").str.replace(")", "").str.lower()
    return df

employeeDF = clean_columns(employeeDF)
education_levelDF = clean_columns(education_levelDF)
performanceDF = clean_columns(performanceDF)
rating_levelDF = clean_columns(rating_levelDF)
satisfaction_levelDF = clean_columns(satisfaction_levelDF)

change the employee columns data types

In [20]:
# Numeric columns
num_cols = ['age','distancefromhome_km','education','salary','stockoptionlevel',
            'yearsatcompany','yearsinmostrecentrole','yearssincelastpromotion','yearswithcurrmanager']

employeeDF[num_cols] = employeeDF[num_cols].apply(pd.to_numeric, errors='coerce')
# Dates
employeeDF['hiredate'] = pd.to_datetime(employeeDF['hiredate'], errors='coerce')


change the performance columns data types

In [24]:
performance_numeric = ['environmentsatisfaction','jobsatisfaction','relationshipsatisfaction',
                       'trainingopportunitieswithinyear','trainingopportunitiestaken',
                       'worklifebalance','selfrating','managerrating']

performanceDF[performance_numeric] = performanceDF[performance_numeric].apply(pd.to_numeric, errors='coerce')
performanceDF['reviewdate'] = pd.to_datetime(performanceDF['reviewdate'], errors='coerce')

remove the spaces and capitalize the first letter

In [41]:
cat_cols_employee = ['firstname','lastname','gender','businesstravel','department','state','ethnicity',
                     'educationfield','jobrole','maritalstatus','overtime','attrition']


cat_cols_education = ['educationlevel']

# cat_cols_performance = ['environmentsatisfaction',
#        'jobsatisfaction', 'relationshipsatisfaction',
#        'trainingopportunitieswithinyear', 'trainingopportunitiestaken',
#        'worklifebalance', 'selfrating', 'managerrating']

cat_cols_rating = ['ratinglevel']

cat_cols_satisfaction = ['satisfactionlevel']

In [48]:
def clean_categorical(df, cat_cols):
     df[cat_cols] = df[cat_cols].apply(lambda x: x.str.strip().str.title())
     return df

employeeDF = clean_categorical(employeeDF, cat_cols_employee)
education_levelDF = clean_categorical(education_levelDF, cat_cols_education)
# performanceDF = clean_categorical(performanceDF, cat_cols_performance)
rating_levelDF = clean_categorical(rating_levelDF, cat_cols_rating)
satisfaction_levelDF = clean_categorical(satisfaction_levelDF, cat_cols_satisfaction)



remove duplicates from the employee table

In [49]:
employeeDF.drop_duplicates(subset='employeeid', inplace=True)

insert the median for every missing value, drop the row of the missing employee id

In [50]:
for col in num_cols:
    employeeDF[col].fillna(employeeDF[col].median(), inplace=True)

# Drop rows with missing IDs
employeeDF.dropna(subset=['employeeid'], inplace=True)

In [51]:
employeeDF.to_csv("../data/clean/employee_clean.csv", index=False)
performanceDF.to_csv("../data/clean/performance_clean.csv", index=False)
education_levelDF.to_csv("../data/clean/education_level_clean.csv", index=False)
rating_levelDF.to_csv("../data/clean/rating_level_clean.csv", index=False)
satisfaction_levelDF.to_csv("../data/clean/satisfaction_clean.csv", index=False)



In [22]:
# needs to be revised
# Step 1: Merge Employee with EducationLevel (many:1)
bigDF = employeeDF.merge(
    education_levelDF, 
    left_on='Education', 
    right_on='EducationLevelID',
    how='left'
)

# Step 2: Merge Performance with Employee (1:many)
bigDF = bigDF.merge(
    performanceDF,
    on='EmployeeID',
    how='left'
)

# Step 3: Merge SatisfactionLevel for EnvironmentSatisfaction, JobSatisfaction, RelationshipSatisfaction
bigDF = bigDF.merge(
    satisfaction_levelDF,
    left_on='EnvironmentSatisfaction',
    right_on='SatisfactionID',
    how='left',
    suffixes=('', '_Env')
)

bigDF = bigDF.merge(
    satisfaction_levelDF,
    left_on='JobSatisfaction',
    right_on='SatisfactionID',
    how='left',
    suffixes=('', '_Job')
)

bigDF = bigDF.merge(
    satisfaction_levelDF,
    left_on='RelationshipSatisfaction',
    right_on='SatisfactionID',
    how='left',
    suffixes=('', '_Rel')
)

# Step 4: Merge RatingLevel for SelfRating and ManagerRating
bigDF = bigDF.merge(
    rating_levelDF,
    left_on='SelfRating',
    right_on='RatingID',
    how='left',
    suffixes=('', '_Self')
)

bigDF = bigDF.merge(
    rating_levelDF,
    left_on='ManagerRating',
    right_on='RatingID',
    how='left',
    suffixes=('', '_Manager')
)

bigDF.to_csv("../data/clean/bigTable.csv", index=False)

In [23]:
# List of EmployeeIDs you want
employee_ids = ['324B-F4DF', '8369-ED26', '1D32-9620']

# Select rows where EmployeeID is in the list
selected_data = bigDF[bigDF['EmployeeID'].isin(employee_ids)]

# Show the result
print(selected_data)


     EmployeeID FirstName  LastName             Gender  Age BusinessTravel  \
4883  1D32-9620  Eldredge    Probyn               Male   23    Some Travel   
4884  1D32-9620  Eldredge    Probyn               Male   23    Some Travel   
4885  1D32-9620  Eldredge    Probyn               Male   23    Some Travel   
5206  8369-ED26    Randie  Tolliday  Prefer Not To Say   25     No Travel    
5207  8369-ED26    Randie  Tolliday  Prefer Not To Say   25     No Travel    
5208  8369-ED26    Randie  Tolliday  Prefer Not To Say   25     No Travel    
6891  324B-F4DF    Hestia   Fishlee             Female   20     No Travel    

           Department  DistanceFromHome (KM) State                  Ethnicity  \
4883  Human Resources                     39    CA                     Other    
4884  Human Resources                     39    CA                     Other    
4885  Human Resources                     39    CA                     Other    
5206       Technology                     19    NY 